# Dual Numbers, Meet Neural Networks

In this [IJulia] Jupyter Notebook, we'll be exploring how **dual numbers** can help use understand (artificial) neural networks: how they are trained, and how they work once trained. All the code in this notebook is written in the [Julia language].

[IJulia]: https://github.com/JuliaLang/IJulia.jl
[Julia language]: http://julialang.org

## Dual numbers

If you are familiar with complex numbers, then dual numbers will be pretty familiar. You can think of them as an extension of the real numbers: every dual number has the form
$$
z = a + b \cdot \varepsilon,
$$
where I'll refer to $a$ as the real component and $b$ as the epsilon component from here on.
The magic arises from the following property of $\varepsilon$: $\varepsilon^2 = 0$.
From this property we can define all sorts of operations on dual numbers, and thats exactly what the package [DualNumbers.jl](https://github.com/JuliaDiff/DualNumbers.jl) does for the `Dual` type that it defines:

In [ ]:
using DualNumbers
z = Dual(2,3)
@show z
@show real(z)
@show epsilon(z)
y = Dual(4,5)
@show y
@show z + y
@show z * y;

In the last example, we can see that the $3\varepsilon \times 5\varepsilon$ term disappeared, as expected.

Dual numbers have various uses, but the one we are interested here is that it enables [automatic diferentiation](http://www.autodiff.org/) of functions. We'll skip the math ([see Wikipedia for a decent if brief explanation](https://en.wikipedia.org/wiki/Dual_number#Differentiation)), but loosely: if we pass a dual number $x + \varepsilon$ instead of a real number $x$ into a function $f$, we not only get the value of the function $f(x)$ but also *the first derivative at $x$*. In particular, the result will also be a dual number, and the epsilon component of that dual number is the derivative. Consider the following simple examples:

In [6]:
square(x) = x^2
@show square(5.0)
@show square(Dual(5.0,1.0))
@show 2.0*5.0

@show sin(0.75π)
@show sin(Dual(0.75π,1))
@show cos(0.75π);

square(5.0) = 25.0
square(Dual(5.0,1.0)) = 25.0 + 10.0du
2.0 * 5.0 = 10.0
sin(0.75π) = 0.7071067811865476
sin(Dual(0.75π,1)) = 0.7071067811865476 - 0.7071067811865475du
cos(0.75π) = -0.7071067811865475


What is happening here is that we are defining a Julia function `square` that takes any input, and tries to square it. When we pass in a `Float64` (`5.0`), Julia just-in-time compiles a version of the function for `Float64`s. The neat thing is that it will do the same thing for `Dual`s too! We can even look at the code that is generated:

In [4]:
@code_native square(5.0)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[2]
Source line: 1
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 1
	vmulsd	%xmm0, %xmm0, %xmm0
	popq	%rbp
	ret


The `mulsd` is essentially saying take the input, and multiply it with itself.

In [5]:
@code_native square(Dual(5.0,1.0))

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[2]
Source line: 1
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 1
	vmovsd	(%rdi), %xmm0
	vmovsd	8(%rdi), %xmm1
Source line: 1
	vaddsd	%xmm1, %xmm1, %xmm1
	vmulsd	%xmm1, %xmm0, %xmm1
	vmulsd	%xmm0, %xmm0, %xmm0
	popq	%rbp
	ret


Here we add the second input to itself (`1+1=2`), then multiply it by the first input (`2*5=10`, the derivative), then multiply the first input by itself (`5*5=25`, the value). So essentially, the minimum possible code. This means using dual numbers in Julia can a very efficient way of getting derivatives without any extra work on the users part. To demonstrate this, consider the following more complicated example:

In [7]:
function squareroot(a)
    x = a
    while abs(x*x - a) > 1e-10
        x = (x + a/x)/2
    end
    return x
end
@show squareroot(9.0)
@show squareroot(Dual(9.0,1.0))
# f (x) = x^0.5
# f'(x) = 0.5*x^-0.5
@show 0.5*9^-0.5 ;

squareroot(9.0) = 3.0
squareroot(Dual(9.0,1.0)) = 3.0 + 0.16666666666666669du
0.5 * 9 ^ -0.5 = 0.16666666666666666


## Training Neural Networks with Dual Numbers

Today we are going to going to consider the problem of *binary classification*. Suppose we have a set of cases $(x_1,y_1),(x_2,y_2),\dots$ where each $x$ is a vector of length $n$ and $y$ is either 0 or 1. We'd like a function $f(x)$ that tells us whether $x$ is a 0 or a 1, and we'd like to learn a function that does this from the data we have. An example application would the important problem of **cat detection**: given an image, which we describe as a series of numbers describing the color of each pixel, is a cat present in the image or not?

Artificial neural networks (ANNs) are, for the purposes of this notebook and very briefly, a particular family of these functions. If you aren't familiar with them, check out the [Wikipedia page](https://en.wikipedia.org/wiki/Artificial_neural_network) or this [EBook](http://neuralnetworksanddeeplearning.com/index.html). We have a *layers* of *neurons*, where a neuron is a simple, and usually nonlinear, function of its weighted inputs. Thus, if we fix the structure of our ANN, we are then optimizing over a family of possible functions, where these functions are defined by these weights. Thus, given weights $w$ and an input $x$, my neural network is the function $g(w,x)$ that gives me a number. We're doing classification, so this number will between 0 and 1 and we'll round it to determine it is class 0 or class 1.

To explore neural networks, we'll first create some random data.

In [12]:
# We'll consider a dimension-2 input space:
#        (1,1)
#    +----+
#    |\ 1 |
#    | \  |
#    |  \ |
#    | 0 \|
#    +----+
# (0,0)
# Where class 0s are below the diagonal, and 1s are above
m = 50  # Number of training samples
srand(1988)  # So we always get same data
X = rand(m,2)  # m rows, n columns, in [0,1]
# Generate class labels
y = [(X[i,1] + X[i,2] <= 1 ? 0.0 : 1.0) for i in 1:m];

We can now layout our neural network. It'll be very simple:

* 2 inputs.
* 4 hidden neurons. Each input connects to each hidden neuron, and each neuron is a sigmoidal function of its weighted inputs.
* 1 output neuron, also a sigmoidal function of its weighted inputs.

We thus need to find values for 12 weights.
Lets make a function that calculates a classification, given weights and an input.

In [8]:
sigmoid(x) = 1/(1 + exp(-x))
# w: 12x1 vector of weights
# x:  2x1 vector of inputs
function ann(w,x)
    # Calculate value at hidden neurons
    h1 = sigmoid(dot(w[1:2],x))
    h2 = sigmoid(dot(w[3:4],x))
    h3 = sigmoid(dot(w[5:6],x))
    h4 = sigmoid(dot(w[7:8],x))
    # Calculate and return value at output
    return sigmoid(dot(w[9:12],[h1,h2,h3,h4]))
end

ann (generic function with 1 method)

In [9]:
# Test out our new ANN with some random weights
srand(1988)
w = rand(12) - 0.5  # Between [-0.5, +0.5]
@show ann(w,[0.0,0.0])
@show round(ann(w,[0.0,0.0]))  # 0 -> Correct :D
@show ann(w,[1.0,1.0])
@show round(ann(w,[1.0,1.0]))  # 0 -> Not correct D:
;

ann(w,[0.0,0.0]) = 0.3044973857244263
round(ann(w,[0.0,0.0])) = 0.0
ann(w,[1.0,1.0]) = 0.28431412765544134
round(ann(w,[1.0,1.0])) = 0.0


How do we find these weights? We must first formalize what it is we are trying to achieve. In this example, we'll consider *square loss*. In particular, given an input $x$, a true class $y$, and the neural networks guess $z$, the error for this input is $(y-z)^2$. The total error across all inputs is the sum of this quantity across all our cases.

In [10]:
# w: 12x1 vector of weights
# X:  mx2 matrix of inputs
# y:  mx1 vector of true labels
function annerror(w,X,y)
    err = 0.0
    for i in 1:length(y)
        z = ann(w,vec(X[i,:]))
        err += (y[i] - z)^2
    end
    return err
end

annerror (generic function with 1 method)

In [13]:
# Test out error with some random weights
srand(1988)
w = rand(12) - 0.5
@show annerror(w,X,y)
w = rand(12) - 0.5
@show annerror(w,X,y)
w = rand(12) - 0.5
@show annerror(w,X,y);

annerror(w,X,y) = 13.983915288773288
annerror(w,X,y) = 12.488382535614333
annerror(w,X,y) = 13.898796784329502


With a notion of error, we now have something we can minimize. "Real" neural networks use *backpropogation* to do this, often coupled with *stochastic gradient descent*. To learn about these things, I'd suggest [this ebook](http://neuralnetworksanddeeplearning.com/index.html). Here we're just going to treat the error of our ANN as a plain old nonlinear function, and minimize that error with respect to the cases and weights using *gradient descent*. In gradient descent, we start at some $x$ and calculate the derivative at $x$, $f^\prime(x)$. We then update $x$ by setting it equal to $x-\lambda f^\prime(x)$, where $\lambda$ is a "small" number. Intutively, the derivate points in the "direction" that the function is increasing in, so by moving in the opposite direction we should decrease the value of our function. Fortunately, we have a pretty handy way of calculating derivatives: **dual numbers**!

What we need to do is to evaluate the neural network 12 times with dual number weights. In each of the 12 calculations most of the weights will just be plain numbers, except one which will have a epsilon component - thats the one we are calculating the derivative with respect to.

In [14]:
# w: 12x1 vector of weights
# X:  mx2 matrix of inputs
# y:  mx1 vector of true labels
function annderiv(w,X,y)
    err = 0.0
    deriv = zeros(12)
    for j in 1:12
        # Create a new w that is dual numbers
        # but with 0 epsilon component...
        w_dual = [Dual(w_k) for w_k in w]
        # Except for the jth component
        w_dual[j] = Dual(w[j],1)
        # Calculate the error for this w,
        # with respect to w_j
        err_j = annerror(w_dual,X,y)
        # Extract the derivative and error
        deriv[j] = epsilon(err_j)
        err = real(err_j)  # This doesn't change with j
    end
    return err, deriv
end

annderiv (generic function with 1 method)

In [15]:
srand(1988)
w = rand(12) - 0.5
@show annerror(w,X,y)
err, deriv = annderiv(w,X,y)
@show err  # matches
deriv

annerror(w,X,y) = 13.983915288773288

12-element Array{Float64,1}:
  0.275391
  0.27482 
  0.344183
  0.343772
  0.397583
  0.395535
  0.386477
  0.38205 
 -0.864485
 -2.41978 
 -1.94487 
 -2.1693  


err = 13.983915288773288


We can calculate derivatives, so all thats left is to "train" our neural network!

In [16]:
# w: 12x1 vector of weights (initial guess)
# X:  mx2 matrix of inputs
# y:  mx1 vector of true labels
# λ: learning rate
# τ: tolerance for stopping
function anntrain(w,X,y;λ=0.2,τ=1e-3)
    err, deriv = annderiv(w,X,y)
    println("Initial error: ", err)
    while true
        # Take step
        w -= λ*deriv
        # Caclulate new error, derivative
        new_err, new_deriv = annderiv(w,X,y)
        if new_err > err
            # We got worse!
            println("Stopping, error got worse! ", new_err)
            break
        elseif abs(new_err - err) <= τ
            # About the same
            println("Stopping, error didn't change enough! ", new_err)
            break
        end
        err, deriv = new_err, new_deriv
    end
    println("Final error: ", err)
    return err, w
end

anntrain (generic function with 1 method)

In [17]:
srand(1988)
init_w = rand(12) - 0.5
opt_err, opt_w = anntrain(init_w,X,y)
@show opt_err
@show opt_w;

Initial error: 13.983915288773288
Stopping, error got worse! 3.394513196126495
Final error: 3.3781551765954303
opt_err = 3.3781551765954303
opt_w = [-1.3558493302790593,-1.2135224028308582,1.3790390467470208,1.2433303868712935,-1.3010050172611491,-1.2881918785307962,0.705890863547658,1.159733839452436,-8.932529517332856,3.151012786161526,-6.278927931383467,0.8465181644415757]


In [18]:
# Lets see how our network does in terms of accuracy
ann_y = Float64[ann(opt_w,vec(X[i,:])) for i in 1:m]
ann_y = round(ann_y)
correct = sum(ann_y .== y)
accuracy = correct / m

0.98

Our neural network correctly classifies 98% of the input samples. However, the real test is how well it performs on data it has never seen before (*out-of-sample testing*).

In [19]:
srand(1234)  # So we always get same data, but not the same as train
X_test = rand(m,2)  # m rows, n columns, in [0,1]
y_test = [(X_test[i,1] + X_test[i,2] <= 1 ? 0.0 : 1.0) for i in 1:m]
ann_y_test = Float64[ann(opt_w,vec(X_test[i,:])) for i in 1:m]
ann_y_test = round(ann_y_test)
correct_test = sum(ann_y_test .== y_test)
accuracy_test = correct_test / m

0.96

We get 96% accuracy on fresh samples, which is pretty good - it means our neural network has probably *generalized* fairly well.

## Interpreting Neural Networks with Dual Numbers

We have so far being using dual numbers on the weights, but there is no reason we can't use dual numbers on the inputs as well. For example, consider calculating the derivative with respect to the inputs in the center of the input space:

In [21]:
ann(opt_w,[Dual(0.5,1), Dual(0.5)])

0.4509053507079046 + 1.0604056636074148du

In [22]:
ann(opt_w,[Dual(0.5), Dual(0.5,1)])

0.4509053507079046 + 1.0051736585860678du

This makes sense: when we are in the center, increasing either component of the input will make us more like class 1.
You may have seen [Google's "inceptionism"](http://googleresearch.blogspot.com/2015/06/inceptionism-going-deeper-into-neural.html) - we can use dual numbers in the same way here. Consider a "random point" that we want to make more like class 1 - what direction should we go in?

In [23]:
anninputderiv(w,x) = [
    epsilon(ann(w,[Dual(x[1],1),Dual(x[2])])),
    epsilon(ann(w,[Dual(x[1]),Dual(x[2],1)]))]

anninputderiv (generic function with 1 method)

In [24]:
x1 = [0.2,0.3]
@show ann(opt_w,x1)
d1 = anninputderiv(opt_w,x1)
@show d1;

ann(opt_w,x1) = 0.06551264206809532
d1 = [0.3492604450563003,0.3301696511678158]


In [25]:
x2 = x1 + 0.5d1
@show x2
@show ann(opt_w,x2)
d2 = anninputderiv(opt_w,x2)
@show d2;

x2 = [0.37463022252815015,0.4650848255839079]
ann(opt_w,x2) = 0.28584906572626806
d2 = [0.976891112109046,0.9248478101055944]


In [26]:
x3 = x2 + 0.5d2
@show x3
@show ann(opt_w,x3)
d3 = anninputderiv(opt_w,x3)
@show d3;

x3 = [0.8630757785826731,0.9275087306367051]
ann(opt_w,x3) = 0.901051341059385
d3 = [0.1871854954638273,0.17865325553691935]


In [27]:
x4 = x3 + 0.5d3
@show x4
@show ann(opt_w,x4)
d4 = anninputderiv(opt_w,x4)
@show d4;

x4 = [0.9566685263145868,1.0168353584051648]
ann(opt_w,x4) = 0.9276051137579988
d4 = [0.11598601972701265,0.11093992991268177]


We've completely changed the class by following the dual number derivatives - from a definite 0, to a definite 1.